In [ ]:
# brew install graphviz
#%pip install graphviz
from graphviz import Digraph

In [ ]:
from micrograd.engine import Value

In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
# a very simple example
x = Value(1.0)
y = (x * 2 + 1).relu()
y.backward()
#draw_dot(y)

In [ ]:
# a simple 2D neuron
import random
from micrograd import nn

random.seed(1337)
n = nn.Neuron(2)
x = [Value(1.0), Value(-2.0)]
y = n(x)
y.backward()

#dot = draw_dot(y)
#dot

In [ ]:
from micrograd.nn import MLP
n = MLP(3,[4,4,1])

In [ ]:
xs = [
    [2.0, 3.0,-1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0,1.0],
    [1.0, 1.0,-1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [n(x) for x in xs]
ypred

In [ ]:
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

In [ ]:
loss.backward()

In [ ]:
#draw_dot(loss)

In [ ]:
#dot.render('gout')

Training av very simple model: y = wx + b
Using micrograd MLP and a dataset of 2 samples

In [ ]:
# Instantiating a 1-layer MLP and setting its parameters
n = MLP(1,[1])   # 1 input, 1 output, y = wx + b
params = n.parameters()
params[0].data = 1.9
params[1].data = 2.1
for p in params:
    print(p.data)
print(n)

# Loading the dataset consisting 2 samples
xs = [[2.0],
      [3.0]]
ys = [6.0,8.0]

In [ ]:
# Generating predictions
ypred = [n(x) for x in xs]
print([yp.data for yp in ypred])

In [ ]:
# Calculating the loss
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

In [ ]:
n.zero_grad()
loss.backward()
learning_rate = 0.01
for p in n.parameters():
    p.data -= p.grad * learning_rate
    print(p.data)

In [ ]:
#draw_dot(loss)

In [ ]:
for i in range(10000):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    n.zero_grad()
    loss.backward()
    for p in n.parameters():
        p.data -= p.grad * learning_rate
    if i % 1000 == 0:
        print(loss, params[0].data, params[1].data)

Training av very simple model: y = wx + b
Using micrograd and a dataset of 2 samples

In [ ]:
# Dataset
x = [[Value(2.0)],
     [Value(3.0)]]
y = [Value(6.0),
     Value(8.0)]

# Weights initialization
w = [Value(1.9)]
b = Value(2.1)

In [ ]:

ypred = [sum((wi*xi for wi,xi in zip(w, xt)), b) for xt in x]
ypred

In [ ]:
# Calculating the loss
loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))
print(f'loss: {loss.data} | grad: {loss.grad}')

In [ ]:
# Reset gradients
for wi in w:
    wi.grad = 0
b.grad = 0

# Backpropagation - calculating gradients
loss.backward()

# Updating weights
learning_rate = 0.01
for wi in w:
    wi.data -= wi.grad * learning_rate
b.data -= b.grad * learning_rate
print(f'w:{[wi.data for wi in w]}, b:{b.data}')

In [ ]:
for i in range(1000):
    ypred = [sum((wi*xi for wi,xi in zip(w, xt)), b) for xt in x]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))
    for wi in w:
        wi.grad = 0
    b.grad = 0
    loss.backward()
    for wi in w:
        wi.data -= wi.grad * learning_rate
    b.data -= b.grad * learning_rate
    if i % 100 == 0:
        print(f'loss: {loss.data} w:{[wi.data for wi in w]}, b:{b.data}')